In [1]:
import numpy as np
import pandas as pd
import csv
import fileinput


In [2]:
rawfile = 'data_act_01.csv'
rawdf = pd.read_csv(rawfile, sep=';')

Como primer punto analizamos los tipos de datos para realizar las conversiones necesarias

In [3]:
rawdf.dtypes

CrimeId                    int64
OriginalCrimeTypeName     object
OffenseDate               object
CallTime                  object
CallDateTime              object
Disposition               object
Address                   object
City                      object
State                     object
AgencyId                  object
Range                    float64
AddressType               object
dtype: object

Al intentar convertir el campo AgendyId al tipo número, encontramos que existen valores no numéricos:

In [4]:
uniqueAgencyValues = rawdf['AgencyId'].unique()
print('Elementos en el campo "AgencyId" que estámos intentando convertir a número: ')
print(uniqueAgencyValues)

Elementos en el campo "AgencyId" que estámos intentando convertir a número: 
['1' 'CA']


Por lo que hacemos una búsqueda de los renglones que generarían este error para intentar corregirlos:

In [5]:
rawdf[pd.to_numeric(rawdf['AgencyId'], errors='coerce').isnull()]

,CrimeId,OriginalCrimeTypeName,OffenseDate,CallTime,CallDateTime,Disposition,Address,City,State,AgencyId,Range,AddressType
5771,160942112,Auto Boost / Strip,2016-04-03T00:00:00,14:30,2016-04-03T14:30:00,REP,Martin Luther King Dr/bowling Green Dr,NaN,NaN,CA,NaN,1
8021,160952280,Auto Boost / Strip,2016-04-04T00:00:00,14:46,2016-04-04T14:46:00,REP,Martin Luther King Dr/nancy Pelosi Dr,S,NaN,CA,NaN,1
8473,160953118,Auto Boost / Strip,2016-04-04T00:00:00,18:11,2016-04-04T18:11:00,REP,Conservatory Drive E/john F Kennedy Dr,NaN,NaN,CA,NaN,1


Una vez que obtenemos los registros que tendrían problema, revisamos estos renglones directamente en el archivo, como texto plano, para identificar el problema que tiene esos renglones y buscar una solución:

In [6]:
nullrows = list(map(lambda x : x + 1, rawdf.index[rawdf['State'].isnull()].tolist()))
with open(rawfile) as flatfile:
    reader=csv.reader(flatfile)
    reviewrows=[row for idx, row in enumerate(reader) if idx in (nullrows)]

reviewrows

[['160942112;Auto Boost / Strip;2016-04-03T00:00:00;14:30;2016-04-03T14:30:00;REP;Martin Luther King Dr/bowling Green Dr;;;CA;;1'],
 ['160952280;Auto Boost / Strip;2016-04-04T00:00:00;14:46;2016-04-04T14:46:00;REP;Martin Luther King Dr/nancy Pelosi Dr; S;;CA;;1'],
 ['160953118;Auto Boost / Strip;2016-04-04T00:00:00;18:11;2016-04-04T18:11:00;REP;Conservatory Drive E/john F Kennedy Dr;;;CA;;1']]

Una vez indentificados los renglones con problemas, y comparando los valores con las columnas o incluso con otros renglones, podemos ver que el problema se presenta porque el formato y número de columnas no es el adecuado, así que reemplazamos el valor incorrecto por un formato adecuado, para corregir los renglones.

In [7]:
searchfor = ';;CA;;1'
replacewith = ';CA;1;;'

with fileinput.FileInput(rawfile, inplace=True, backup='.bak') as file:
    for line in file:
        print(line.replace(searchfor, replacewith), end='')

Corregidos los renglones, volvemos a cargar el archivo ya preparado:

In [8]:
prepared = pd.read_csv(rawfile, sep=';')

In [9]:
nullrowsN = list(map(lambda x: x-1, nullrows))
prepared.iloc[nullrowsN]

,CrimeId,OriginalCrimeTypeName,OffenseDate,CallTime,CallDateTime,Disposition,Address,City,State,AgencyId,Range,AddressType
5771,160942112,Auto Boost / Strip,2016-04-03T00:00:00,14:30,2016-04-03T14:30:00,REP,Martin Luther King Dr/bowling Green Dr,NaN,CA,1,NaN,NaN
8021,160952280,Auto Boost / Strip,2016-04-04T00:00:00,14:46,2016-04-04T14:46:00,REP,Martin Luther King Dr/nancy Pelosi Dr,S,CA,1,NaN,NaN
8473,160953118,Auto Boost / Strip,2016-04-04T00:00:00,18:11,2016-04-04T18:11:00,REP,Conservatory Drive E/john F Kennedy Dr,NaN,CA,1,NaN,NaN


Al revisar las columnas con valores NaN, encontramos que es común que el campo "City" y "Range" contengan valores NaN, por lo que los dejamos tal cual.

Pero el campo "AdressType" no es común que tenga valores NaN, por lo que analizaremos los valores más adecuados para este campo.

In [10]:
prepared['AddressType'].value_counts()

Premise Address    5059
Intersection       3701
Common Location     818
Geo-Override        469
Intersectioon         1
Name: AddressType, dtype: int64

Establecemos la moda como valor de los valores NaN:

In [11]:
prepared['AddressType'].fillna('Premise Address', inplace=True)

Al revisar los valores, podemos ver que existe solo un registro con el valor "Intersectioon", haciendo obvio que es un error tipográfico, por lo que hacemos la corrección:

In [12]:
prepared['AddressType'].replace({'Intersectioon': 'Intersection'}, inplace=True)

Al corregir los renglones con problema, también nos encontramos que el campo "City" tiene algunos valores que no parecen ser los correctos y hacemos la correción:

In [13]:
prepared['City'].value_counts()

San Francisco    9665
Treasure Isla      51
Daly City           5
Presidio            3
Yerba Buena         3
 S                  1
Brisbane            1
SAN FRANCISCO       1
Name: City, dtype: int64

In [14]:
prepared['City'].replace({'SAN FRANCISCO': 'San Francisco', ' S': 'San Francisco'}, inplace=True)

Ya que se han corregido los registros con problemas, volvemos a revisar los valores para comprobar que la fuente de datos esté preparada.

In [15]:
prepared.iloc[nullrowsN]

,CrimeId,OriginalCrimeTypeName,OffenseDate,CallTime,CallDateTime,Disposition,Address,City,State,AgencyId,Range,AddressType
5771,160942112,Auto Boost / Strip,2016-04-03T00:00:00,14:30,2016-04-03T14:30:00,REP,Martin Luther King Dr/bowling Green Dr,NaN,CA,1,NaN,Premise Address
8021,160952280,Auto Boost / Strip,2016-04-04T00:00:00,14:46,2016-04-04T14:46:00,REP,Martin Luther King Dr/nancy Pelosi Dr,San Francisco,CA,1,NaN,Premise Address
8473,160953118,Auto Boost / Strip,2016-04-04T00:00:00,18:11,2016-04-04T18:11:00,REP,Conservatory Drive E/john F Kennedy Dr,NaN,CA,1,NaN,Premise Address


Como revisión adicional, revisamos el campo "Disposition", el cual comparamos contra la lista "San Francisco Police Deparment - Disposition codes" obtenida desde Kaggle. Cargamos la lista de códigos y comparamos la existencia de los códigos en la lista para obtener los códigos que no existen:

In [16]:
sfpddispocodes = pd.read_csv('sfpd_disposition_codes.csv')

In [17]:
nodispocode = prepared.merge(sfpddispocodes, on=['Disposition'], how='left', indicator=True)

nodispocode[nodispocode['_merge'] == 'left_only']['Disposition'].value_counts()

Not recorded    543
INC              17
CRT               2
Name: Disposition, dtype: int64

Una vez encontrados todos los códigos inexistentes, los reemplzamos por el código "ND" que de acuerdo a la lista, corresponden a "No Disposition":

In [18]:
prepared['Disposition'].replace({'Not recorded': 'ND', 'INC': 'ND', 'CRT': 'ND'}, inplace=True)

Ahora sí estamos listos para convertir el campo "AgencyId" a un campo de tipo número y los campos "CallDateTime" y "OffenseDate" al tipo fecha:

In [19]:
prepared['CallDateTime'] = pd.to_datetime(prepared['CallDateTime'], errors='coerce')
prepared['OffenseDate'] = pd.to_datetime(prepared['OffenseDate'], errors='coerce')
prepared['AgencyId'] = pd.to_numeric(prepared['AgencyId'], errors='coerce')

Removemos columnas innecesarias:
La columna "OffenseDate" parecería innecesaria, pero la dejamos porque puede ser que al reporte un crimen de otra fecha distinta a la de la llamada recibida.
La columna "CallTime" es innecesaria porque el valor del tiempo ya está incluido en la columna "CallDateTime".
La columna "Range" no contiene ningún valor, por lo que se considera innecesaria.

In [20]:
prepared.drop(columns=['CallTime', 'Range'], inplace=True)

In [21]:
prepared.dtypes

CrimeId                           int64
OriginalCrimeTypeName            object
OffenseDate              datetime64[ns]
CallDateTime             datetime64[ns]
Disposition                      object
Address                          object
City                             object
State                            object
AgencyId                          int64
AddressType                      object
dtype: object

La primer columna "CrimeId" debería ser el campo llave de esta fuente de datos, por lo que revisamos si contiene valores únicos:

In [22]:
prepared[prepared.duplicated(subset=['CrimeId'],keep=False)]

,CrimeId,OriginalCrimeTypeName,OffenseDate,CallDateTime,Disposition,Address,City,State,AgencyId,AddressType
26,160913455,Vandalism,2016-03-31,2016-03-31 20:53:00,ND,1600 Block Of Sunnydale Av,San Francisco,CA,1,Premise Address
1707,160913455,Susp,2016-04-01,2016-04-01 18:29:00,GOA,Geary St/larkin St,San Francisco,CA,1,Intersection
3792,160913455,Passing Call,2016-04-02,2016-04-02 17:11:00,ND,900 Block Of Market St,San Francisco,CA,1,Premise Address
7045,160950496,Passing Call,2016-04-04,2016-04-04 06:51:00,HAN,University St/felton St,San Francisco,CA,1,Intersection
7046,160950496,Suspicious Vehicle,2016-04-04,2016-04-04 06:51:00,ND,1400 Block Of Cabrillo St,San Francisco,CA,1,Premise Address
7047,160950496,Trespasser,2016-04-04,2016-04-04 06:51:00,CAN,Block Of Hampshire St,San Francisco,CA,1,Premise Address


In [23]:
duplicated = prepared.index[prepared.duplicated(subset=['CrimeId'],keep=False)].tolist()
max = prepared['CrimeId'].max()

for element in duplicated:
  prepared.iloc[element]['CrimeId'] = max + element;

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Finalmente, con todos los datos preparados y convertidos al formato adecuado, generamos la nueva fuente de datos en formato CSV y en formato JSON.

In [24]:
prepared.to_csv('data_act_01_prepared.csv', sep=',', index=False, header=True)

In [25]:
prepared.to_json('data_act_01_prepared.json', orient='records', lines=True)

In [26]:
prepared.head(10)

,CrimeId,OriginalCrimeTypeName,OffenseDate,CallDateTime,Disposition,Address,City,State,AgencyId,AddressType
0,160903280,Assault / Battery,2016-03-30,2016-03-30 18:42:00,REP,100 Block Of Chilton Av,San Francisco,CA,1,Premise Address
1,160912272,Homeless Complaint,2016-03-31,2016-03-31 15:31:00,GOA,2300 Block Of Market St,San Francisco,CA,1,Premise Address
2,160912590,Susp Info,2016-03-31,2016-03-31 16:49:00,GOA,2300 Block Of Market St,San Francisco,CA,1,Premise Address
3,160912801,Report,2016-03-31,2016-03-31 17:38:00,GOA,500 Block Of 7th St,San Francisco,CA,1,Premise Address
4,160912811,594,2016-03-31,2016-03-31 17:42:00,REP,Beale St/bryant St,San Francisco,CA,1,Intersection
5,160913003,Ref'd,2016-03-31,2016-03-31 18:29:00,GOA,16th St/pond St,San Francisco,CA,1,Intersection
6,160913050,Homeless Complaint,2016-03-31,2016-03-31 18:43:00,ADV,Berwick Pl/harrison St,San Francisco,CA,1,Intersection
7,160913056,Homeless Complaint,2016-03-31,2016-03-31 18:47:00,HAN,Florida St/mariposa St,San Francisco,CA,1,Intersection
8,160913078,Agg Assault / Adw Dv,2016-03-31,2016-03-31 18:52:00,ND,100 Block Of Genebern Wy,San Francisco,CA,1,Premise Address
9,160913103,Encampment,2016-03-31,2016-03-31 18:57:00,ADV,2700 Block Of Folsom St,San Francisco,CA,1,Premise Address
